<a href="https://colab.research.google.com/github/Krishnaugale353/TB_detection/blob/main/tb_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import numpy as np
from pathlib import Path
import cv2
import math
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from collections.abc import Iterable
import glob

In [3]:
batch_size=30
patch_size=16
img_size=224
num_patches=(img_size//patch_size)**2
p_dim=768
heads_att=12
num_encoder=12

In [4]:
target_class={0:"Normal",1:"Tuberculosis"}
Normal_dir=Path("/content/drive/MyDrive/archive/TB_Chest_Radiography_Database/Normal")
TB_dir=Path("/content/drive/MyDrive/archive/TB_Chest_Radiography_Database/Tuberculosis")

In [5]:
Normal_images:list=sorted(list(map(str, list(Normal_dir.glob("*.png")))))
TB_images:list=sorted(list(map(str, list(TB_dir.glob("*.png")))))
Normal_labels:list=[0]*len(Normal_images)
TB_labels:list=[1]*len(TB_images)

In [6]:
images=np.array(Normal_images+TB_images)
labels=np.array(Normal_labels+TB_labels)
images.shape, labels.shape

((0,), (0,))

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(images,labels,test_size=0.2,random_state=42)
x_valid,x_test,y_valid,y_test=train_test_split(x_valid,y_valid,test_size=0.5,random_state=42)
(x_train.shape,x_valid.shape,x_test.shape)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
def image_preprocessing(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    clahe=cv2.createCLAHE(clipLimit=2)
    img=clahe.apply(img)
    img=cv2.GaussianBlur(img,(5,5),0,borderType=cv2.BORDER_CONSTANT)
    img=cv2.resize(img,(img_size,img_size),interpolation=cv2.INTER_LINEAR)
    img=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    img=np.moveaxis(img,-1,0)
    return img

In [ ]:
!nvidia-smi

In [ ]:
x_train=np.array(list(map(image_preprocessing,x_train))).astype(np.float32)
x_valid=np.array(list(map(image_preprocessing,x_valid))).astype(np.float32)
x_test=np.array(list(map(image_preprocessing,x_test))).astype(np.float32)
x_train.shape,x_valid.shape,x_test.shape

In [ ]:
device=("cuda" if torch.cuda.is_available() else "cpu")
x_train=torch.from_numpy(x_train).to(device)
x_valid=torch.from_numpy(x_valid).to(device)
x_test=torch.from_numpy(x_test).to(device)
y_train=torch.from_numpy(y_train).to(device)
y_valid=torch.from_numpy(y_valid).to(device)
y_test=torch.from_numpy(y_test).to(device)

In [ ]:
class StandardScaler():
    def __init__(self) -> None:
        self.mean=None
        self.std=None
    def fit(self,tensor:torch.Tensor) -> None:
        self.mean=tensor.mean((0,2,3),keepdim=True)
        self.std=tensor.std((0,2,3),keepdim=True)
    def transform(self,tensor:torch.Tensor) -> torch.Tensor:
        scaled=(tensor-self.mean)/(self.std+1e-5)
        return scaled
    def fit_transform(self,tensor:torch.Tensor) -> torch.Tensor:
        self.fit(tensor=tensor)
        scaled=self.transform(tensor=tensor)
        return scaled

In [ ]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
x_valid=scaler.transform(x_valid)

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
train_dataset=TensorDataset(x_train,y_train)
val_dataset=TensorDataset(x_valid,y_valid)
test_dataset=TensorDataset(x_test,y_test)
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)
val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

In [8]:
def extract_image_patches(x:torch.Tensor, kernel:int, stride:int=1, dilation:int=1):

    b,c,h,w = x.shape
    h2 = math.ceil(h / stride)
    w2 = math.ceil(w / stride)
    pad_row = (h2 - 1) * stride + (kernel - 1) * dilation + 1 - h
    pad_col = (w2 - 1) * stride + (kernel - 1) * dilation + 1 - w
    x = torch.nn.functional.pad(x, (pad_row//2, pad_row - pad_row//2, pad_col//2, pad_col - pad_col//2))

    # Extract patches
    patches = x.unfold(2, kernel, stride).unfold(3, kernel, stride)
    patches = patches.permute(0, 2, 3, 1, 4, 5).contiguous()
    patches = patches.view(*patches.size()[:3], -1)

    #flatten patches
    return patches.view(b,-1,patches.shape[-1])

In [9]:
class EncoderBlock(torch.nn.Module):
    def __init__(self, projection_dim:int, num_heads:int, num_patches:int):
        super().__init__()
        self.projection_dim=projection_dim
        self.num_heads=num_heads
        self.num_patches=num_patches

        self.norm1=torch.nn.LayerNorm(self.projection_dim)
        self.norm2=torch.nn.LayerNorm(self.projection_dim)
        self.attention=torch.nn.MultiheadAttention(self.projection_dim,self.num_heads,batch_first=True)
        self.mlp=torch.nn.Sequential(
            torch.nn.Linear(self.projection_dim,self.projection_dim*4),
            torch.nn.GELU(),
            torch.nn.Linear(self.projection_dim*4,self.projection_dim),
        )

    def forward(self,x):
        x1=self.norm1(x)
        attention=self.attention(x1,x1,x1)[0]
        x2=attention+x
        x3=self.norm2(x2)
        x3=self.mlp(x3)
        out=x2+x3
        return out

In [10]:
class PEG(torch.nn.Module):
    def __init__(self,dim:int,k:int=3):
        super().__init__()
        self.pos=torch.nn.Conv2d(dim,dim,k,1,1,groups=dim)
    def forward(self,x,H,W):
        B,N,C=x.shape
        feat_tokens=x
        feat_tokens=feat_tokens.transpose(1,2).view(B,C,H,W)
        x=self.pos(feat_tokens)+feat_tokens
        x=x.flatten(2).transpose(1,2)
        return x

In [11]:
class ViT(torch.nn.Module):
    def __init__(self,
                 patch_size:int,
                 num_patches:int,
                 projection_dim:int,
                 num_heads:int,
                 num_encoder:int):
        super().__init__()
        self.patch_size=patch_size
        self.num_patches=num_patches
        self.projection_dim=projection_dim
        self.num_heads=num_heads
        self.num_encoder=num_encoder

        self.input_d=self.patch_size*self.patch_size*3
        self.linear=torch.nn.Linear(self.input_d,self.projection_dim)
        self.peg=PEG(self.projection_dim,3)
        self.blocks = torch.nn.ModuleList([
            EncoderBlock(self.projection_dim, self.num_heads, self.num_patches) for _ in range(self.num_encoder)
        ])
        self.ln_out=torch.nn.LayerNorm(self.projection_dim)

        self.out=torch.nn.Sequential(
            torch.nn.Linear(self.projection_dim,1),
            torch.nn.Sigmoid()
        )
    def forward(self,images:torch.Tensor):
        batch_size=images.size()[0]
        x=extract_image_patches(images,self.patch_size,self.patch_size)
        x=self.linear(x)
        encoded=x

        for i,block in enumerate(self.blocks):
            encoded=block(encoded)
            if i==0:
                _H,_W=int(self.num_patches**0.5),int(self.num_patches**0.5)
                encoded=self.peg(encoded,_H,_W)

        rep=encoded.mean(dim=1)
        rep=self.ln_out(rep)
        output=self.out(rep)
        return output

In [12]:
model=ViT(patch_size=patch_size,
          num_patches=num_patches,
          projection_dim=p_dim,
          num_heads=heads_att,
          num_encoder=num_encoder).to(device)
print(model)

NameError: name 'device' is not defined

In [13]:
vit_total_params = sum(p.numel() for p in model.parameters())
vit_total_params

NameError: name 'model' is not defined

In [14]:
epoch=30
loss_fn=torch.nn.BCELoss()
opt=torch.optim.SGD(model.parameters(),lr=1e-3,momentum=0.9)
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=opt,T_max=epoch,verbose=True)

NameError: name 'model' is not defined

In [ ]:
best_loss = float('inf')
best_model_weights = None
patience=10
for i in range(epoch):
    print(f"Epoch: {i+1}\n")
    model.train()
    train_loss=0
    train_batches=len(train_loader)
    for batch,(x,y) in enumerate(train_loader):
        pred=model(x).squeeze(-1)
        loss=loss_fn(pred,y.float())
        train_loss+=loss.item()
        loss.backward()
        opt.step()
        opt.zero_grad()
    train_loss/=train_batches
    print(f"Train Error: \n Avg loss: {train_loss:>8f} \n")
    model.eval()
    test_loss=0
    test_batches=len(val_loader)
    with torch.no_grad():
        for x,y in val_loader:
            pred=model(x).squeeze(-1)
            test_loss += loss_fn(pred,y.float()).item()
    test_loss /= test_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    if test_loss<best_loss:
        best_loss=test_loss
        best_model_weights=model.state_dict()
        patience=10
    else:
        patience-=1
        if patience==0:
            break
    scheduler.step()
model.load_state_dict(best_model_weights)
print(f'best loss:', best_loss)

Epoch: 1

Train Error: 
 Avg loss: 0.404695 

Test Error: 
 Avg loss: 0.298038 

Adjusting learning rate of group 0 to 9.9975e-04.
Epoch: 2

Train Error: 
 Avg loss: 0.314925 

Test Error: 
 Avg loss: 0.259149 

Adjusting learning rate of group 0 to 9.9901e-04.
Epoch: 3

Train Error: 
 Avg loss: 0.272572 

Test Error: 
 Avg loss: 0.309277 

Adjusting learning rate of group 0 to 9.9778e-04.
Epoch: 4

Train Error: 
 Avg loss: 0.208800 

Test Error: 
 Avg loss: 0.161119 

Adjusting learning rate of group 0 to 9.9606e-04.
Epoch: 5

Train Error: 
 Avg loss: 0.162049 

Test Error: 
 Avg loss: 0.160920 

Adjusting learning rate of group 0 to 9.9384e-04.
Epoch: 6

Train Error: 
 Avg loss: 0.124916 

Test Error: 
 Avg loss: 0.111742 

Adjusting learning rate of group 0 to 9.9114e-04.
Epoch: 7

Train Error: 
 Avg loss: 0.106986 

Test Error: 
 Avg loss: 0.119696 

Adjusting learning rate of group 0 to 9.8796e-04.
Epoch: 8

Train Error: 
 Avg loss: 0.088546 

Test Error: 
 Avg loss: 0.094396 

Ad

In [1]:
model.eval()
val_preds=[]
with torch.no_grad():
    for x,y in val_loader:
        pred=model(x).squeeze(-1)
        val_preds.append(pred.cpu().numpy())
val_preds=np.concatenate(val_preds,axis=0)
val_preds=np.round(val_preds)
recall_score(y_valid.cpu().numpy(),val_preds),accuracy_score(y_valid.cpu().numpy(),val_preds)

NameError: name 'model' is not defined

In [ ]:
model.eval()
test_preds=[]
with torch.no_grad():
    for x,y in test_loader:
        pred=model(x).squeeze(-1)
        test_preds.append(pred.cpu().numpy())
test_preds=np.concatenate(test_preds,axis=0)
test_preds=np.round(test_preds)
recall_score(y_test.cpu().numpy(),test_preds),accuracy_score(y_test.cpu().numpy(),test_preds)

In [ ]:
def evaluation_parametrics(name,y_val, y_pred):

    print("\n------------------------{}------------------------\n".format(name))

    cm_test = confusion_matrix(y_val, y_pred)
    t1 = ConfusionMatrixDisplay(cm_test)
    print("\nClassification Report\n")
    print(classification_report(y_val, y_pred))
    print("--------------------------------------------------------------------------")

    t1.plot()
evaluation_parametrics("Confusion Matrix - Validation Dataset", y_valid.cpu().numpy(), val_preds)